# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import datetime
from optionalyzer.options import Call, Put
from optionalyzer.blackscholes import BlackScholes
from optionalyzer.chart import PayoffChart

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_dark"

# Payoff Chart

A payoff chart in options is a graph where on x axis we have the price of the underlying asset and on y axis we have the payoff of the option. The payoff of the option is the profit or loss that we make if we exercise the option at a particular price of the underlying asset. The payoff chart is a very useful tool to understand the behavior of the option. It helps us to understand the risk and reward profile of the option. It also helps us to understand the break even points of the option.

Now, the payoff chart is build for a particular time. Let's built one!

## Payoff Chart for a Call Option

We'll use TSLA options. We'll chose a call option expirying on 2023-04-21. Let's use the strike price of 120, which is at the money. (TSLA is currently trading at 113.)

In [2]:
bs = BlackScholes()

In [3]:
K = 120
expiry_date = "2023-04-21"
c = Call(K, expiry_date)
c

Call(120, 2023-04-21)

In [4]:
today = datetime.date.today().strftime("%Y-%m-%d")
today

'2023-01-07'

Let's calculate payoff chart for today.

In [5]:
risk_free_rate = 0.0342
sigma = 0.6
S = 113
bought_price = c.calculate_price(S, risk_free_rate, sigma, today=today)
bought_price

12.010075109702946

What will happen if we excercised the option today?

In [6]:
S = np.arange(100,130,0.1)
prices = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=today, return_greeks=False)
iv = c.intrinsic_value(S)
pnl = prices - bought_price

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=S, y=prices, name="Price"))
fig.add_trace(go.Scatter(x=S, y=iv, name="Intrinsic Value"))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

let's see the PNL if we excercised the option on 2023-02-21.

In [7]:
S = np.arange(100,140,0.1)
new_today = "2023-02-21"
prices = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
iv = c.intrinsic_value(S)
pnl = prices - bought_price

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=S, y=prices, name="Price"))
fig.add_trace(go.Scatter(x=S, y=iv, name="Intrinsic Value"))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

And at the last date.

In [8]:
S = np.arange(100,140,0.1)
new_today = "2023-04-21"
prices = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
iv = c.intrinsic_value(S)
pnl = prices - bought_price

ps_pnl_mask = np.argwhere(pnl>=0)
ng_pnl_mask = np.argwhere(pnl<0)

ps_pnl = pnl[ps_pnl_mask].flatten()
ng_pnl = pnl[ng_pnl_mask].flatten()

ps_S = S[ps_pnl_mask].flatten()
ng_S = S[ng_pnl_mask].flatten()

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=S, y=prices, name="Price"))
fig.add_trace(go.Scatter(x=S, y=iv, name="Intrinsic Value"))
fig.add_trace(go.Scatter(x=ps_S, y=ps_pnl, fill="tozeroy", fillcolor="rgba(0,255,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=ng_S, y=ng_pnl, fill="tozeroy", fillcolor="rgba(255,0,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

## Payoff Chart for a Call + Put

We'll use TSLA options. We'll chose a call option expirying on 2023-04-21. Let's use the strike price of 120, which is at the money. (TSLA is currently trading at 113.)

In [9]:
K = 115
expiry_date = "2023-04-21"
c = Call(K, expiry_date)
p = Put(K, expiry_date)
c, p

(Call(115, 2023-04-21), Put(115, 2023-04-21))

In [10]:
today = datetime.date.today().strftime("%Y-%m-%d")
today

'2023-01-07'

Let's calculate payoff chart for today.

In [11]:
risk_free_rate = 0.0342
sigma = 0.6
S = 113
bought_price_c = c.calculate_price(S, risk_free_rate, sigma, today=today)
bought_price_p = p.calculate_price(S, risk_free_rate, sigma, today=today)
bought_price_c, bought_price_p

(13.994752345347585, 14.87955914156521)

What will happen if we excercised the option today?

In [12]:
S = np.arange(100,130,0.1)
prices_c = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=today, return_greeks=False)
prices_p = p.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=today, return_greeks=False)
pnl = prices_c + prices_p - bought_price_c - bought_price_p

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

let's see the PNL if we excercised the option on 2023-02-21.

In [13]:
S = np.arange(100,140,0.1)
new_today = "2023-02-21"
prices_c = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
prices_p = p.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
pnl = prices_c + prices_p - bought_price_c - bought_price_p

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

And at the last date.

In [14]:
S = np.arange(100,140,0.1)
new_today = "2023-04-21"
prices_c = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
prices_p = p.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
pnl = prices_c + prices_p - bought_price_c - bought_price_p

ps_pnl_mask = np.argwhere(pnl>=0)
ng_pnl_mask = np.argwhere(pnl<0)

ps_pnl = pnl[ps_pnl_mask].flatten()
ng_pnl = pnl[ng_pnl_mask].flatten()

ps_S = S[ps_pnl_mask].flatten()
ng_S = S[ng_pnl_mask].flatten()

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=ps_S, y=ps_pnl, fill="tozeroy", fillcolor="rgba(0,255,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=ng_S, y=ng_pnl, fill="tozeroy", fillcolor="rgba(255,0,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

## Payoff Chart for a Call - Put

We'll use TSLA options. We'll chose a call option expirying on 2023-04-21. Let's use the strike price of 120, which is at the money. (TSLA is currently trading at 113.)

In [15]:
K = 115
expiry_date = "2023-04-21"
c = Call(K, expiry_date)
p = Put(K, expiry_date)
c, p

(Call(115, 2023-04-21), Put(115, 2023-04-21))

In [16]:
today = datetime.date.today().strftime("%Y-%m-%d")
today

'2023-01-07'

Let's calculate payoff chart for today.

In [17]:
risk_free_rate = 0.0342
sigma = 0.6
S = 113
bought_price_c = c.calculate_price(S, risk_free_rate, sigma, today=today)
bought_price_p = p.calculate_price(S, risk_free_rate, sigma, today=today)
bought_price_c, bought_price_p

(13.994752345347585, 14.87955914156521)

What will happen if we excercised the option today?

In [18]:
S = np.arange(100,130,0.1)
prices_c = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=today, return_greeks=False)
prices_p = p.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=today, return_greeks=False)
pnl = prices_c - prices_p - bought_price_c + bought_price_p
ps_pnl_mask = np.argwhere(pnl>=0)
ng_pnl_mask = np.argwhere(pnl<0)

ps_pnl = pnl[ps_pnl_mask].flatten()
ng_pnl = pnl[ng_pnl_mask].flatten()

ps_S = S[ps_pnl_mask].flatten()
ng_S = S[ng_pnl_mask].flatten()

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=ps_S, y=ps_pnl, fill="tozeroy", fillcolor="rgba(0,255,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=ng_S, y=ng_pnl, fill="tozeroy", fillcolor="rgba(255,0,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

let's see the PNL if we excercised the option on 2023-02-21.

In [19]:
S = np.arange(100,140,0.1)
new_today = "2023-02-21"
prices_c = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
prices_p = p.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
pnl = prices_c - prices_p - bought_price_c + bought_price_p

ps_pnl_mask = np.argwhere(pnl>=0)
ng_pnl_mask = np.argwhere(pnl<0)

ps_pnl = pnl[ps_pnl_mask].flatten()
ng_pnl = pnl[ng_pnl_mask].flatten()

ps_S = S[ps_pnl_mask].flatten()
ng_S = S[ng_pnl_mask].flatten()

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=ps_S, y=ps_pnl, fill="tozeroy", fillcolor="rgba(0,255,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=ng_S, y=ng_pnl, fill="tozeroy", fillcolor="rgba(255,0,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()

And at the last date.

In [20]:
S = np.arange(100,140,0.1)
new_today = "2023-04-21"
prices_c = c.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
prices_p = p.calculate_price(S, interest_rate=risk_free_rate, volatility=sigma, today=new_today, return_greeks=False)
pnl = prices_c - prices_p - bought_price_c + bought_price_p

ps_pnl_mask = np.argwhere(pnl>=0)
ng_pnl_mask = np.argwhere(pnl<0)

ps_pnl = pnl[ps_pnl_mask].flatten()
ng_pnl = pnl[ng_pnl_mask].flatten()

ps_S = S[ps_pnl_mask].flatten()
ng_S = S[ng_pnl_mask].flatten()

fig = go.Figure()
fig.add_hline(0, line_dash="dash", line_color="white", name="Break Even")
fig.add_trace(go.Scatter(x=ps_S, y=ps_pnl, fill="tozeroy", fillcolor="rgba(0,255,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=ng_S, y=ng_pnl, fill="tozeroy", fillcolor="rgba(255,0,0,0.5)", name=""))
fig.add_trace(go.Scatter(x=S, y=pnl, name="PnL"))
fig.update_layout(title="Payoff Chart", xaxis_title="Stock Price", yaxis_title="Price")
fig.show()